Install packages

Add ifind

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64/'
os.environ['JVM_PATH'] = os.environ['JAVA_HOME'] + 'lib/server/libjvm.so'

In [3]:
!export PYTHONPATH="${PYTHONPATH}:./ifind"b

Prepare topics

Run simulations

In [10]:
#!rm /workspace/sims/output/*

In [ ]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_nyt_bm25.xml

In [1]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_wapo_bm25.xml

Traceback (most recent call last):
  File "run_simiir.py", line 50, in <module>
    main(sys.argv[1])
  File "run_simiir.py", line 19, in main
    for configuration in config_reader:
  File "/workspace/simiir/config_readers/simulation_config_reader.py", line 59, in __next__
    bg = SimulationComponentGenerator(self._config_dict['@id'], configuration_set)
  File "/workspace/simiir/../simiir/config_readers/component_generators/simulation_generator.py", line 28, in __init__
    self.search_interface = self._get_object_reference(config_details=self._config_dict['searchInterface'],
  File "/workspace/simiir/config_readers/component_generators/base_generator.py", line 50, in _get_object_reference
    available_classes = self.__get_available_classes(package)
  File "/workspace/simiir/config_readers/component_generators/base_generator.py", line 95, in __get_available_classes
    module_references.append(importlib.import_module(module))
  File "/usr/local/lib/python3.8/importlib/__init__.py", 

In [6]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_wapo_monoT5.xml

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
SIMULATION 'wapo_monoT5-321-GPT+_perfect_10rpp'
  Simulation Configuration:
    Topic: 321
    Search Interface: PyterrierWebSearchInterface
        index_dir: 
        corpus: wapo
        neural: _monot5

  User Configuration (GPT+_perfect_10rpp):
    Query Generator: PredeterminedQueryGenerator
        stopword_file: /workspace/data/stopword-list.txt
        query_file: /workspace/data/wapo/gpt+.txt
        user: 1
    Snippet Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.0
        data_handler: 0
    Document Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.0
        data_handler: 0
    Stopping Decision Maker: FixedDep

# Evaluations

## Modalities

### sDCG 

## Browsing behavior

### sDCG